<a href="https://colab.research.google.com/github/batmanvane/complex-systems-modeling/blob/main/notebooks/08_chaos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complex Systems Modeling - Session 8

# Introduction to Chaos

In previous sessions, we looked at how systems settle into stable states (equilibria). Today, we explore what happens when they **don't** settle down. We will investigate **Chaos**: a behavior that looks random but is actually governed by deterministic rules.

We will explore:
1.  **The Logistic Map:** How simple population growth models turn chaotic.
2.  **Bifurcation Diagrams:** Visualizing the transition from order to chaos.
3.  **Sensitivity to Initial Conditions:** The "Butterfly Effect".
4.  **The Lorenz System:** Chaos in continuous time (3D weather models).

---

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, VBox, HBox, Label
from IPython.display import display
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

print("Libraries imported successfully!")

## Part 1: Chaos in Discrete Time - The Logistic Map

The Logistic Map is one of the most famous examples of how a simple equation can produce complex behavior. It models population growth with a resource limit:

$$ x_{t} = r \cdot x_{t-1} (1 - x_{t-1}) $$

* $x_t$: Population at time $t$ (between 0 and 1)
* $r$: Growth rate parameter

Let's visualize the time series and the "Cobweb Plot" to see how the system behaves as we change $r$.

In [ ]:
def logistic_map(x, r):
    return r * x * (1 - x)

def plot_logistic_interactive(r, x0, steps):
    # Initialize arrays
    t = np.arange(steps)
    x = np.zeros(steps)
    x[0] = x0

    # Iterate map
    for i in range(1, steps):
        x[i] = logistic_map(x[i-1], r)

    # Create Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # --- 1. Time Series Plot ---
    ax1.plot(t, x, 'b.-', linewidth=1)
    ax1.set_ylim(0, 1)
    ax1.set_xlabel('Time step (t)')
    ax1.set_ylabel('Population (x)')
    ax1.set_title(f'Time Series (r={r})')
    ax1.grid(True, alpha=0.3)

    # --- 2. Cobweb Plot ---
    # Plot the parabola y = r*x*(1-x)
    xi = np.linspace(0, 1, 100)
    yi = logistic_map(xi, r)
    ax2.plot(xi, yi, 'k-', linewidth=2, label='Map')
    ax2.plot([0, 1], [0, 1], 'g--', linewidth=1, label='y=x') # Diagonal

    # Plot the cobweb trajectory
    cx, cy = [x0], [0]
    for i in range(steps-1):
        # Move vertically to the curve
        cx.append(x[i])
        cy.append(x[i+1])
        # Move horizontally to the diagonal
        cx.append(x[i+1])
        cy.append(x[i+1])

    ax2.plot(cx, cy, 'r-', alpha=0.6, linewidth=1)
    ax2.plot(x0, 0, 'ro', label='Start')

    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)
    ax2.set_xlabel('$x_t$')
    ax2.set_ylabel('$x_{t+1}$')
    ax2.set_title('Cobweb Plot')
    ax2.legend(loc='upper left')

    plt.show()

print("="*70)
print("LOGISTIC MAP EXPLORER")
print("="*70)
print("Try these r values:")
print("  r < 1.0 : Extinction (Population -> 0)")
print("  r = 2.5 : Stable Equilibrium")
print("  r = 3.2 : Period-2 Oscillation")
print("  r = 3.5 : Period-4 Oscillation")
print("  r > 3.6 : Chaos!")

slider_r = FloatSlider(min=0.0, max=4.0, step=0.01, value=2.5, description='r (growth):', continuous_update=False)
slider_x0 = FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, description='x0 (start):', continuous_update=False)
slider_steps = IntSlider(min=20, max=200, step=10, value=50, description='steps:', continuous_update=False)

w1 = interactive(plot_logistic_interactive, r=slider_r, x0=slider_x0, steps=slider_steps)
display(w1)

## Part 2: The Bifurcation Diagram

Instead of looking at just one trajectory, let's look at the **long-term behavior** for *all* values of $r$ at once.

A **Bifurcation Diagram** plots the parameter $r$ on the x-axis and the stable states (asymptotic values) of $x$ on the y-axis.

* **Single line:** System settles to a Fixed Point.
* **Split (Pitchfork):** System oscillates (Period Doubling).
* **Cloud of points:** Chaos.

In [ ]:
def draw_bifurcation_diagram(r_min, r_max, steps_to_discard, steps_to_keep):
    r_values = np.linspace(r_min, r_max, 1000)
    x = 0.5 * np.ones_like(r_values) # Initial condition array

    plt.figure(figsize=(12, 8))

    # Iterate to get past transient behavior
    for _ in range(steps_to_discard):
        x = logistic_map(x, r_values)

    # Iterate and plot asymptotic behavior
    for _ in range(steps_to_keep):
        x = logistic_map(x, r_values)
        # We plot small dots with low alpha (transparency) to see density
        plt.plot(r_values, x, ',k', alpha=0.1)

    plt.title(f'Bifurcation Diagram of Logistic Map')
    plt.xlabel('Growth rate (r)')
    plt.ylabel('Asymptotic Population (x)')
    plt.xlim(r_min, r_max)
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.show()

print("Generating Bifurcation Diagram (this may take a few seconds)...")
# Focus on the interesting region r=[2.5, 4.0]
draw_bifurcation_diagram(2.5, 4.0, 200, 200)

## Part 3: Sensitivity to Initial Conditions (The Butterfly Effect)

The defining characteristic of chaos is that two trajectories starting extremely close together will diverge exponentially fast.

Let's simulate two populations:
1.  $x_{start} = x_0$
2.  $x_{start} = x_0 + 0.000001$

Watch what happens when $r$ is in the chaotic regime (e.g., $r=3.9$).

In [ ]:
def plot_sensitivity(r, x0_base, steps):
    epsilon = 0.000001 # A tiny difference
    x1 = x0_base
    x2 = x0_base + epsilon

    history1 = [x1]
    history2 = [x2]
    diff = [abs(x1-x2)]

    for _ in range(steps):
        x1 = logistic_map(x1, r)
        x2 = logistic_map(x2, r)
        history1.append(x1)
        history2.append(x2)
        diff.append(abs(x1-x2))

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

    # Plot Trajectories
    ax1.plot(history1, 'b-', alpha=0.7, label=f'Start: {x0_base}')
    ax1.plot(history2, 'r--', alpha=0.7, label=f'Start: {x0_base} + {epsilon}')
    ax1.set_ylabel('Population')
    ax1.set_title(f'Sensitivity to Initial Conditions (r={r})')
    ax1.legend()
    ax1.grid(True)

    # Plot Difference
    ax2.plot(diff, 'k-', label='Difference |x1 - x2|')
    ax2.set_ylabel('Difference')
    ax2.set_xlabel('Time Step')
    ax2.set_yscale('log') # Log scale to see exponential growth
    ax2.legend()
    ax2.grid(True)

    plt.show()

slider_r_sens = FloatSlider(min=2.5, max=4.0, step=0.01, value=3.9, description='r:', continuous_update=False)

w2 = interactive(plot_sensitivity, r=slider_r_sens, x0_base=FloatSlider(value=0.2, min=0.1, max=0.9), steps=IntSlider(value=60, min=20, max=150))
display(w2)

## Part 4: Chaos in Continuous Time - The Lorenz System

In continuous time (differential equations), chaos requires at least 3 dimensions. The most famous example is the **Lorenz System**, developed by Edward Lorenz to model atmospheric convection.

$$ \frac{dx}{dt} = \sigma(y - x) $$
$$ \frac{dy}{dt} = x(\rho - z) - y $$
$$ \frac{dz}{dt} = xy - \beta z $$

Let's visualize the famous "Strange Attractor" in 3D.

In [ ]:
def lorenz_system(state, t, sigma, rho, beta):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

def plot_lorenz(sigma, rho, beta, duration):
    dt = 0.01
    t = np.arange(0, duration, dt)
    initial_state = [1.0, 1.0, 1.0]

    # Solve ODE
    states = odeint(lorenz_system, initial_state, t, args=(sigma, rho, beta))

    # 3D Plot
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(states[:, 0], states[:, 1], states[:, 2], linewidth=0.7)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f'Lorenz Attractor (rho={rho})')
    plt.show()

print("="*70)
print("LORENZ ATTRACTOR 3D VISUALIZER")
print("="*70)
print("Standard Chaos parameters: sigma=10, rho=28, beta=8/3 (approx 2.67)")

style = {'description_width': 'initial'}
s_sigma = FloatSlider(value=10.0, min=0, max=50, description='Sigma (Prandtl):', style=style)
s_rho = FloatSlider(value=28.0, min=0, max=50, description='Rho (Rayleigh):', style=style)
s_beta = FloatSlider(value=2.67, min=0, max=10, description='Beta:', style=style)
s_dur = IntSlider(value=50, min=10, max=100, description='Duration:', style=style)

w3 = interactive(plot_lorenz, sigma=s_sigma, rho=s_rho, beta=s_beta, duration=s_dur)
display(w3)